# Chapter 46: Building an E-commerce Store

E-commerce applications present unique challenges: complex state management across sessions, secure payment processing, inventory synchronization, and high-concurrency checkout flows. This chapter demonstrates how to build a production-ready online store using Next.js Server Actions for secure transactions, Stripe for payment processing, optimistic UI patterns for cart management, and edge caching for product catalogs.

By the end of this chapter, you'll architect a scalable product catalog with faceted search, implement persistent shopping carts with optimistic updates, integrate Stripe for secure payment processing, build inventory management systems to prevent overselling, create order tracking and admin dashboards, and optimize checkout flows for conversion rate optimization.

## 46.1 Product Catalog Setup

Designing a performant product discovery experience with faceted navigation.

### Database Schema for E-commerce

```typescript
// prisma/schema.prisma
model Product {
  id          String   @id @default(cuid())
  slug        String   @unique
  name        String
  description String   @db.Text
  price       Decimal  @db.Decimal(10, 2)
  comparePrice Decimal? @db.Decimal(10, 2)
  sku         String   @unique
  inventory   Int      @default(0)
  lowStock    Int      @default(5)
  status      ProductStatus @default(DRAFT)
  images      ProductImage[]
  variants    ProductVariant[]
  categories  Category[]
  attributes  ProductAttribute[]
  reviews     Review[]
  orderItems  OrderItem[]
  
  // SEO
  metaTitle   String?
  metaDesc    String?
  
  // Tracking
  salesCount  Int      @default(0)
  viewCount   Int      @default(0)
  
  createdAt   DateTime @default(now())
  updatedAt   DateTime @updatedAt

  @@index([status, categories])
  @@index([slug])
  @@index([price])
  @@fulltext([name, description])
}

model ProductVariant {
  id        String  @id @default(cuid())
  productId String
  product   Product @relation(fields: [productId], references: [id], onDelete: Cascade)
  sku       String  @unique
  name      String  // e.g., "Large / Red"
  price     Decimal @db.Decimal(10, 2)
  inventory Int     @default(0)
  options   Json    // { size: "L", color: "red" }
  
  @@index([productId])
}

model ProductImage {
  id        String  @id @default(cuid())
  productId String
  product   Product @relation(fields: [productId], references: [id], onDelete: Cascade)
  url       String
  alt       String?
  order     Int     @default(0)
  
  @@index([productId])
}

model Category {
  id          String    @id @default(cuid())
  name        String
  slug        String    @unique
  description String?
  parentId    String?
  parent      Category? @relation("CategoryHierarchy", fields: [parentId], references: [id])
  children    Category[] @relation("CategoryHierarchy")
  products    Product[]
  
  @@index([slug])
}

enum ProductStatus {
  DRAFT
  ACTIVE
  ARCHIVED
}
```

### Product Listing with Faceted Search

```typescript
// app/shop/page.tsx
import { Suspense } from 'react';
import { ProductGrid } from '@/features/product-grid';
import { FilterSidebar } from '@/features/filters';
import { prisma } from '@/shared/lib/db';
import { unstable_cache } from 'next/cache';

interface ShopPageProps {
  searchParams: {
    category?: string;
    minPrice?: string;
    maxPrice?: string;
    sort?: 'price-asc' | 'price-desc' | 'newest';
    page?: string;
  };
}

const getCachedProducts = unstable_cache(
  async (params: ShopPageProps['searchParams']) => {
    const where = {
      status: 'ACTIVE' as const,
      inventory: { gt: 0 },
      ...(params.category && {
        categories: { some: { slug: params.category } },
      }),
      ...(params.minPrice || params.maxPrice) && {
        price: {
          ...(params.minPrice && { gte: parseFloat(params.minPrice) }),
          ...(params.maxPrice && { lte: parseFloat(params.maxPrice) }),
        },
      },
    };

    const orderBy = {
      ...(params.sort === 'price-asc' && { price: 'asc' as const }),
      ...(params.sort === 'price-desc' && { price: 'desc' as const }),
      ...(params.sort === 'newest' && { createdAt: 'desc' as const }),
      ...(!params.sort && { salesCount: 'desc' as const }),
    };

    const page = parseInt(params.page || '1');
    const limit = 12;
    const skip = (page - 1) * limit;

    const [products, total, categories] = await Promise.all([
      prisma.product.findMany({
        where,
        include: {
          images: { take: 1, orderBy: { order: 'asc' } },
          categories: { select: { name: true, slug: true } },
        },
        orderBy,
        skip,
        take: limit,
      }),
      prisma.product.count({ where }),
      prisma.category.findMany({
        where: { parentId: null },
        include: { children: true },
      }),
    ]);

    return { products, total, pages: Math.ceil(total / limit), categories };
  },
  ['products-list'],
  { revalidate: 60, tags: ['products'] }
);

export default async function ShopPage({ searchParams }: ShopPageProps) {
  const { products, total, pages, categories } = await getCachedProducts(searchParams);

  return (
    <div className="container mx-auto py-8 px-4">
      <div className="flex flex-col lg:flex-row gap-8">
        <aside className="w-full lg:w-64 flex-shrink-0">
          <FilterSidebar categories={categories} />
        </aside>
        
        <main className="flex-1">
          <div className="mb-4 flex justify-between items-center">
            <p className="text-gray-600">{total} products</p>
            <SortDropdown />
          </div>
          
          <Suspense fallback={<ProductGridSkeleton />}>
            <ProductGrid products={products} />
          </Suspense>
          
          <Pagination currentPage={parseInt(searchParams.page || '1')} totalPages={pages} />
        </main>
      </div>
    </div>
  );
}
```

### Real-time Inventory Check

Prevent overselling with inventory validation:

```typescript
// features/cart/actions.ts
'use server';

import { prisma } from '@/shared/lib/db';
import { revalidateTag } from 'next/cache';

export async function checkInventory(productId: string, quantity: number, variantId?: string) {
  const product = await prisma.product.findUnique({
    where: { id: productId },
    include: { variants: variantId ? { where: { id: variantId } } : false },
  });

  if (!product || product.status !== 'ACTIVE') {
    return { available: false, message: 'Product unavailable' };
  }

  const availableStock = variantId 
    ? product.variants[0]?.inventory ?? 0
    : product.inventory;

  if (availableStock < quantity) {
    return { 
      available: false, 
      message: `Only ${availableStock} items available`,
      availableStock 
    };
  }

  return { available: true, availableStock };
}

export async function reserveInventory(productId: string, quantity: number, variantId?: string) {
  // Optimistic inventory reservation during checkout
  try {
    if (variantId) {
      await prisma.productVariant.update({
        where: { id: variantId },
        data: { inventory: { decrement: quantity } },
      });
    } else {
      await prisma.product.update({
        where: { id: productId },
        data: { inventory: { decrement: quantity } },
      });
    }
    
    revalidateTag('products');
    return { success: true };
  } catch (error) {
    return { success: false, error: 'Insufficient inventory' };
  }
}
```

## 46.2 Shopping Cart Implementation

Persistent cart with optimistic UI updates.

### Cart State Management

```typescript
// stores/cart.ts
import { create } from 'zustand';
import { persist } from 'zustand/middleware';
import { checkInventory } from '@/features/cart/actions';

interface CartItem {
  id: string;
  productId: string;
  variantId?: string;
  name: string;
  price: number;
  quantity: number;
  image: string;
  maxStock: number;
}

interface CartStore {
  items: CartItem[];
  isLoading: boolean;
  error: string | null;
  
  // Actions
  addItem: (item: Omit<CartItem, 'quantity'>, quantity?: number) => Promise<void>;
  removeItem: (id: string) => void;
  updateQuantity: (id: string, quantity: number) => Promise<void>;
  clearCart: () => void;
  getTotal: () => number;
  getItemCount: () => number;
}

export const useCart = create<CartStore>()(
  persist(
    (set, get) => ({
      items: [],
      isLoading: false,
      error: null,

      addItem: async (product, quantity = 1) => {
        const { items } = get();
        const existingItem = items.find(
          i => i.productId === product.productId && i.variantId === product.variantId
        );

        const newQuantity = existingItem ? existingItem.quantity + quantity : quantity;

        // Server-side inventory check
        const check = await checkInventory(product.productId, newQuantity, product.variantId);
        
        if (!check.available) {
          set({ error: check.message || 'Not enough stock' });
          return;
        }

        if (existingItem) {
          set({
            items: items.map(i =>
              i.id === existingItem.id ? { ...i, quantity: newQuantity } : i
            ),
            error: null,
          });
        } else {
          set({
            items: [...items, { ...product, quantity, maxStock: check.availableStock || 999 }],
            error: null,
          });
        }
      },

      updateQuantity: async (id, quantity) => {
        const { items } = get();
        const item = items.find(i => i.id === id);
        if (!item) return;

        if (quantity > item.maxStock) {
          set({ error: `Maximum ${item.maxStock} items available` });
          return;
        }

        if (quantity < 1) {
          set({ items: items.filter(i => i.id !== id) });
        } else {
          set({
            items: items.map(i => (i.id === id ? { ...i, quantity } : i)),
            error: null,
          });
        }
      },

      removeItem: (id) => {
        set({ items: get().items.filter(i => i.id !== id) });
      },

      clearCart: () => set({ items: [], error: null }),
      
      getTotal: () => get().items.reduce((sum, item) => sum + item.price * item.quantity, 0),
      
      getItemCount: () => get().items.reduce((sum, item) => sum + item.quantity, 0),
    }),
    {
      name: 'shopping-cart',
      skipHydration: true, // Handle hydration mismatch
    }
  )
);
```

### Optimistic Cart UI

```typescript
// components/cart/cart-drawer.tsx
'use client';

import { useCart } from '@/stores/cart';
import { useState, useTransition } from 'react';
import Image from 'next/image';
import Link from 'next/link';

export function CartDrawer() {
  const { items, removeItem, updateQuantity, getTotal, error } = useCart();
  const [isPending, startTransition] = useTransition();
  const [optimisticQuantities, setOptimisticQuantities] = useState<Record<string, number>>({});

  const handleUpdateQuantity = (id: string, newQuantity: number) => {
    // Optimistic update
    setOptimisticQuantities(prev => ({ ...prev, [id]: newQuantity }));
    
    startTransition(async () => {
      await updateQuantity(id, newQuantity);
      setOptimisticQuantities(prev => {
        const next = { ...prev };
        delete next[id];
        return next;
      });
    });
  };

  return (
    <div className="w-full max-w-md bg-white h-full flex flex-col">
      <header className="p-4 border-b">
        <h2 className="text-lg font-semibold">Shopping Cart ({items.length})</h2>
      </header>

      {error && (
        <div className="bg-red-50 text-red-600 p-3 text-sm">
          {error}
        </div>
      )}

      <div className="flex-1 overflow-auto p-4 space-y-4">
        {items.map((item) => {
          const displayQuantity = optimisticQuantities[item.id] ?? item.quantity;
          
          return (
            <div key={item.id} className="flex gap-4">
              <div className="relative w-20 h-20 bg-gray-100 rounded">
                <Image
                  src={item.image}
                  alt={item.name}
                  fill
                  className="object-cover rounded"
                />
              </div>
              
              <div className="flex-1">
                <h3 className="font-medium line-clamp-2">{item.name}</h3>
                <p className="text-gray-600">${item.price.toFixed(2)}</p>
                
                <div className="flex items-center gap-2 mt-2">
                  <button
                    onClick={() => handleUpdateQuantity(item.id, displayQuantity - 1)}
                    disabled={isPending}
                    className="w-8 h-8 rounded border flex items-center justify-center hover:bg-gray-50 disabled:opacity-50"
                  >
                    -
                  </button>
                  <span className="w-8 text-center">{displayQuantity}</span>
                  <button
                    onClick={() => handleUpdateQuantity(item.id, displayQuantity + 1)}
                    disabled={isPending || displayQuantity >= item.maxStock}
                    className="w-8 h-8 rounded border flex items-center justify-center hover:bg-gray-50 disabled:opacity-50"
                  >
                    +
                  </button>
                </div>
              </div>
              
              <button
                onClick={() => removeItem(item.id)}
                className="text-gray-400 hover:text-red-500"
              >
                <TrashIcon className="w-5 h-5" />
              </button>
            </div>
          );
        })}
      </div>

      <footer className="p-4 border-t space-y-4">
        <div className="flex justify-between text-lg font-semibold">
          <span>Total</span>
          <span>${getTotal().toFixed(2)}</span>
        </div>
        
        <Link
          href="/checkout"
          className="block w-full bg-black text-white text-center py-3 rounded-lg hover:bg-gray-800 transition"
        >
          Checkout
        </Link>
      </footer>
    </div>
  );
}
```

## 46.3 Stripe Integration

Secure payment processing with Stripe Elements.

### Payment Intent Creation

```typescript
// app/api/stripe/payment-intent/route.ts
import { NextResponse } from 'next/server';
import { stripe } from '@/shared/lib/stripe';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { getServerSession } from 'next-auth';
import { prisma } from '@/shared/lib/db';

export async function POST(request: Request) {
  try {
    const session = await getServerSession(authOptions);
    const body = await request.json();
    const { items, shipping } = body;

    // Calculate amount
    const amount = items.reduce(
      (sum: number, item: any) => sum + item.price * item.quantity * 100, // Convert to cents
      0
    );

    // Create or retrieve customer
    let customerId = session?.user?.stripeCustomerId;
    if (!customerId && session?.user?.email) {
      const customer = await stripe.customers.create({
        email: session.user.email,
        metadata: { userId: session.user.id },
      });
      customerId = customer.id;
      
      // Save to user record
      await prisma.user.update({
        where: { id: session.user.id },
        data: { stripeCustomerId: customerId },
      });
    }

    // Create payment intent
    const paymentIntent = await stripe.paymentIntents.create({
      amount: Math.round(amount),
      currency: 'usd',
      customer: customerId,
      automatic_payment_methods: { enabled: true },
      metadata: {
        items: JSON.stringify(items.map((i: any) => ({ id: i.productId, qty: i.quantity }))),
      },
      shipping: {
        name: shipping.name,
        address: {
          line1: shipping.line1,
          line2: shipping.line2,
          city: shipping.city,
          state: shipping.state,
          postal_code: shipping.postal_code,
          country: shipping.country,
        },
      },
    });

    return NextResponse.json({ clientSecret: paymentIntent.client_secret });
  } catch (error) {
    console.error('Payment intent error:', error);
    return NextResponse.json(
      { error: 'Failed to create payment intent' },
      { status: 500 }
    );
  }
}
```

### Checkout Form with Elements

```typescript
// app/checkout/page.tsx
import { redirect } from 'next/navigation';
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { CheckoutForm } from '@/features/checkout/checkout-form';
import { prisma } from '@/shared/lib/db';

export default async function CheckoutPage() {
  const session = await getServerSession(authOptions);
  
  // Pre-fill user data if authenticated
  const user = session?.user ? await prisma.user.findUnique({
    where: { id: session.user.id },
    include: { addresses: true },
  }) : null;

  return (
    <div className="container mx-auto max-w-2xl py-8 px-4">
      <h1 className="text-2xl font-bold mb-8">Checkout</h1>
      <CheckoutForm user={user} />
    </div>
  );
}

// features/checkout/checkout-form.tsx
'use client';

import { useState } from 'react';
import { loadStripe } from '@stripe/stripe-js';
import { Elements, PaymentElement, useStripe, useElements } from '@stripe/react-stripe-js';
import { useCart } from '@/stores/cart';

const stripePromise = loadStripe(process.env.NEXT_PUBLIC_STRIPE_PUBLISHABLE_KEY!);

export function CheckoutForm({ user }: { user: any }) {
  const [clientSecret, setClientSecret] = useState<string | null>(null);
  const { items, getTotal } = useCart();

  // Create payment intent on load
  useState(() => {
    fetch('/api/stripe/payment-intent', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({
        items,
        shipping: user?.addresses[0], // Simplified
      }),
    })
      .then(res => res.json())
      .then(data => setClientSecret(data.clientSecret));
  });

  if (!clientSecret) return <div>Loading...</div>;

  return (
    <Elements stripe={stripePromise} options={{ clientSecret }}>
      <PaymentForm total={getTotal()} />
    </Elements>
  );
}

function PaymentForm({ total }: { total: number }) {
  const stripe = useStripe();
  const elements = useElements();
  const [isLoading, setIsLoading] = useState(false);
  const [errorMessage, setErrorMessage] = useState<string | null>(null);
  const { clearCart } = useCart();

  const handleSubmit = async (e: React.FormEvent) => {
    e.preventDefault();
    if (!stripe || !elements) return;

    setIsLoading(true);
    setErrorMessage(null);

    const { error, paymentIntent } = await stripe.confirmPayment({
      elements,
      confirmParams: {
        return_url: `${window.location.origin}/checkout/success`,
      },
      redirect: 'if_required',
    });

    if (error) {
      setErrorMessage(error.message || 'Payment failed');
    } else if (paymentIntent.status === 'succeeded') {
      // Clear cart and redirect
      clearCart();
      window.location.href = `/checkout/success?payment_intent=${paymentIntent.id}`;
    }

    setIsLoading(false);
  };

  return (
    <form onSubmit={handleSubmit} className="space-y-6">
      <PaymentElement />
      
      {errorMessage && (
        <div className="bg-red-50 text-red-600 p-3 rounded">
          {errorMessage}
        </div>
      )}

      <button
        type="submit"
        disabled={!stripe || isLoading}
        className="w-full bg-black text-white py-4 rounded-lg font-semibold hover:bg-gray-800 disabled:opacity-50"
      >
        {isLoading ? 'Processing...' : `Pay $${total.toFixed(2)}`}
      </button>
    </form>
  );
}
```

## 46.4 Order Management

Post-purchase flow and order tracking.

### Order Creation Webhook

```typescript
// app/api/stripe/webhook/route.ts
import { NextResponse } from 'next/server';
import { headers } from 'next/headers';
import { stripe } from '@/shared/lib/stripe';
import { prisma } from '@/shared/lib/db';
import { revalidateTag } from 'next/cache';

const endpointSecret = process.env.STRIPE_WEBHOOK_SECRET!;

export async function POST(request: Request) {
  const payload = await request.text();
  const sig = headers().get('stripe-signature')!;

  let event;

  try {
    event = stripe.webhooks.constructEvent(payload, sig, endpointSecret);
  } catch (err: any) {
    return NextResponse.json({ error: `Webhook Error: ${err.message}` }, { status: 400 });
  }

  if (event.type === 'payment_intent.succeeded') {
    const paymentIntent = event.data.object;
    const items = JSON.parse(paymentIntent.metadata.items);
    
    // Create order
    const order = await prisma.order.create({
      data: {
        stripePaymentIntentId: paymentIntent.id,
        status: 'PAID',
        total: paymentIntent.amount / 100,
        currency: paymentIntent.currency,
        customerEmail: paymentIntent.receipt_email,
        shippingAddress: paymentIntent.shipping as any,
        items: {
          create: items.map((item: any) => ({
            productId: item.id,
            quantity: item.qty,
            price: item.price,
          })),
        },
      },
    });

    // Update inventory
    for (const item of items) {
      await prisma.product.update({
        where: { id: item.id },
        data: { 
          inventory: { decrement: item.qty },
          salesCount: { increment: item.qty },
        },
      });
    }

    // Revalidate product pages to show updated inventory
    revalidateTag('products');
    
    // Send confirmation email (async)
    await fetch(`${process.env.NEXT_PUBLIC_URL}/api/emails/order-confirmation`, {
      method: 'POST',
      body: JSON.stringify({ orderId: order.id }),
    });
  }

  return NextResponse.json({ received: true });
}
```

### Order Tracking Page

```typescript
// app/orders/[id]/page.tsx
import { notFound } from 'next/navigation';
import { prisma } from '@/shared/lib/db';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { getServerSession } from 'next-auth';
import { OrderTimeline } from '@/features/orders/order-timeline';

interface Props {
  params: { id: string };
}

export default async function OrderPage({ params }: Props) {
  const session = await getServerSession(authOptions);
  
  const order = await prisma.order.findUnique({
    where: { id: params.id },
    include: {
      items: {
        include: {
          product: {
            select: { name: true, images: { take: 1 } },
          },
        },
      },
    },
  });

  if (!order) notFound();
  
  // Verify ownership
  if (order.userId && order.userId !== session?.user?.id) {
    return <div className="text-center py-12">Access denied</div>;
  }

  return (
    <div className="container mx-auto max-w-4xl py-8 px-4">
      <div className="mb-8">
        <h1 className="text-2xl font-bold">Order #{order.id.slice(-8)}</h1>
        <p className="text-gray-600">Placed on {new Date(order.createdAt).toLocaleDateString()}</p>
      </div>

      <div className="grid md:grid-cols-3 gap-8">
        <div className="md:col-span-2 space-y-6">
          <OrderTimeline status={order.status} />
          
          <div className="border rounded-lg p-6">
            <h2 className="font-semibold mb-4">Items</h2>
            <div className="space-y-4">
              {order.items.map((item) => (
                <div key={item.id} className="flex gap-4">
                  <div className="w-20 h-20 bg-gray-100 rounded relative">
                    {item.product.images[0] && (
                      <Image
                        src={item.product.images[0].url}
                        alt={item.product.name}
                        fill
                        className="object-cover rounded"
                      />
                    )}
                  </div>
                  <div className="flex-1">
                    <h3 className="font-medium">{item.product.name}</h3>
                    <p className="text-gray-600">Qty: {item.quantity}</p>
                    <p className="font-medium">${item.price.toFixed(2)}</p>
                  </div>
                </div>
              ))}
            </div>
          </div>
        </div>

        <div className="space-y-6">
          <div className="border rounded-lg p-6">
            <h2 className="font-semibold mb-4">Order Summary</h2>
            <div className="space-y-2 text-sm">
              <div className="flex justify-between">
                <span>Subtotal</span>
                <span>${(order.total * 0.9).toFixed(2)}</span>
              </div>
              <div className="flex justify-between">
                <span>Tax</span>
                <span>${(order.total * 0.1).toFixed(2)}</span>
              </div>
              <div className="flex justify-between font-bold text-lg pt-2 border-t">
                <span>Total</span>
                <span>${order.total.toFixed(2)}</span>
              </div>
            </div>
          </div>

          <div className="border rounded-lg p-6">
            <h2 className="font-semibold mb-4">Shipping</h2>
            <address className="not-italic text-sm text-gray-600">
              {order.shippingAddress?.name}<br />
              {order.shippingAddress?.line1}<br />
              {order.shippingAddress?.city}, {order.shippingAddress?.state} {order.shippingAddress?.postal_code}
            </address>
          </div>
        </div>
      </div>
    </div>
  );
}
```

## 46.5 Admin Dashboard

Inventory and order management for store owners.

### Admin Layout with Role Guard

```typescript
// app/admin/layout.tsx
import { redirect } from 'next/navigation';
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { AdminNav } from '@/features/admin/admin-nav';

export default async function AdminLayout({ children }: { children: React.ReactNode }) {
  const session = await getServerSession(authOptions);
  
  if (session?.user?.role !== 'ADMIN') {
    redirect('/');
  }

  return (
    <div className="flex h-screen bg-gray-50">
      <aside className="w-64 bg-white border-r">
        <AdminNav />
      </aside>
      <main className="flex-1 overflow-auto p-8">
        {children}
      </main>
    </div>
  );
}

// app/admin/products/page.tsx
import { prisma } from '@/shared/lib/db';
import { ProductTable } from '@/features/admin/product-table';
import { AddProductButton } from '@/features/admin/add-product-button';

export default async function AdminProductsPage() {
  const products = await prisma.product.findMany({
    include: {
      _count: { select: { orderItems: true } },
      categories: true,
    },
    orderBy: { createdAt: 'desc' },
  });

  return (
    <div>
      <div className="flex justify-between items-center mb-6">
        <h1 className="text-2xl font-bold">Products</h1>
        <AddProductButton />
      </div>
      
      <ProductTable products={products} />
    </div>
  );
}
```

### Bulk Inventory Update

```typescript
// app/admin/api/bulk-update/route.ts
import { NextResponse } from 'next/server';
import { prisma } from '@/shared/lib/db';
import { revalidateTag } from 'next/cache';
import { z } from 'zod';

const bulkUpdateSchema = z.array(z.object({
  id: z.string(),
  inventory: z.number().min(0),
  price: z.number().min(0).optional(),
}));

export async function POST(request: Request) {
  try {
    const body = await request.json();
    const updates = bulkUpdateSchema.parse(body);

    // Transactional update
    await prisma.$transaction(
      updates.map(update => 
        prisma.product.update({
          where: { id: update.id },
          data: {
            inventory: update.inventory,
            ...(update.price && { price: update.price }),
          },
        })
      )
    );

    revalidateTag('products');

    return NextResponse.json({ success: true, updated: updates.length });
  } catch (error) {
    return NextResponse.json(
      { error: 'Invalid update data' }, 
      { status: 400 }
    );
  }
}
```

## 46.6 Performance Optimization

E-commerce specific optimizations for conversion.

### Edge Caching for Product Data

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const response = NextResponse.next();

  // Cache product images and API responses at edge
  if (request.nextUrl.pathname.startsWith('/api/products')) {
    response.headers.set(
      'Cache-Control',
      'public, s-maxage=60, stale-while-revalidate=300'
    );
  }

  return response;
}

export const config = {
  matcher: ['/api/products/:path*', '/shop/:path*'],
};
```

### Image Optimization Strategy

```typescript
// components/product/product-image.tsx
import Image from 'next/image';

interface ProductImageProps {
  src: string;
  alt: string;
  priority?: boolean;
}

export function ProductImage({ src, alt, priority = false }: ProductImageProps) {
  return (
    <div className="relative aspect-square overflow-hidden bg-gray-100 rounded-lg">
      <Image
        src={src}
        alt={alt}
        fill
        className="object-cover object-center group-hover:scale-105 transition-transform duration-300"
        sizes="(max-width: 768px) 100vw, (max-width: 1200px) 50vw, 33vw"
        priority={priority}
        quality={85}
      />
    </div>
  );
}
```

### Analytics and Conversion Tracking

```typescript
// lib/analytics/ecommerce.ts
import { sendGAEvent } from '@next/third-parties/google';

export function trackAddToCart(product: {
  id: string;
  name: string;
  price: number;
  quantity: number;
}) {
  sendGAEvent('event', 'add_to_cart', {
    currency: 'USD',
    value: product.price * product.quantity,
    items: [{
      item_id: product.id,
      item_name: product.name,
      price: product.price,
      quantity: product.quantity,
    }],
  });
}

export function trackPurchase(order: {
  id: string;
  total: number;
  items: Array<{ id: string; name: string; price: number; quantity: number }>;
}) {
  sendGAEvent('event', 'purchase', {
    transaction_id: order.id,
    value: order.total,
    currency: 'USD',
    items: order.items.map(item => ({
      item_id: item.id,
      item_name: item.name,
      price: item.price,
      quantity: item.quantity,
    })),
  });
}
```

## Key Takeaways from Chapter 46

1. **Product Catalog Architecture**: Use Prisma with PostgreSQL for relational product data (variants, categories, attributes). Implement faceted search using compound indexes on `(status, categories)` and full-text search on `(name, description)`. Cache product listings with `unstable_cache` tagged for invalidation when inventory updates occur.

2. **Inventory Management**: Prevent overselling by checking stock server-side before adding to cart using Server Actions. Implement optimistic inventory reservation during checkout that decrements stock upon webhook confirmation, with automatic restocking on payment failure. Use database transactions for atomic inventory updates across multiple items.

3. **Shopping Cart State**: Manage cart state with Zustand using the `persist` middleware for localStorage persistence across sessions. Implement optimistic UI patterns that update quantities immediately while validating against server-side inventory checks, showing error states only when constraints are violated.

4. **Stripe Integration**: Create PaymentIntents server-side with calculated totals and metadata for order reconstruction. Use Stripe Elements for PCI-compliant card collection without handling raw card data. Implement webhook handlers for `payment_intent.succeeded` events to create orders, update inventory, and trigger confirmation emails asynchronously.

5. **Order Lifecycle**: Store order data normalized across `Order` and `OrderItem` tables with foreign key relationships to products for historical accuracy. Create protected order tracking pages that verify user ownership or allow guest lookup via email/ID combinations. Use Server Components for order details to prevent client-side data exposure.

6. **Admin Operations**: Build role-protected admin interfaces using layout-level authentication guards checking for `ADMIN` role. Implement bulk update APIs with database transactions for inventory management, and use `revalidateTag` to purge edge caches when product data changes.

7. **Conversion Optimization**: Cache product API routes at the edge with `s-maxage=60, stale-while-revalidate=300` headers to handle flash sales traffic. Optimize product images with Next.js Image component using responsive sizes and priority loading for above-the-fold content. Implement Google Analytics 4 ecommerce events for add-to-cart and purchase tracking to measure conversion funnels.

## Coming Up Next

**Chapter 47: Building a SaaS Dashboard**

Moving beyond transactional e-commerce, Chapter 47 explores multi-tenant SaaS architecture. You'll build a subscription-based dashboard with organization-level data isolation, real-time analytics using Server-Sent Events, role-based access control for team members, Stripe Subscription integration for recurring billing, and feature flags for tiered plan limitations. Learn to architect database schemas for multi-tenancy and implement row-level security patterns for data isolation.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='45. blog_platform.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='47. saas_dashboard.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
